In [1]:
from localmd.decomposition import localmd_decomposition, display, factored_svd, windowed_pmd, single_residual_block_md, get_projector
from localmd.dataset import MultipageTiffDataset
import localmd.visualization 
from localmd.visualization import generate_PMD_comparison_triptych
from localmd.pmdarray import PMDArray

import matplotlib.pyplot as plt


import scipy
import scipy.sparse
import jax
import jax.scipy
import jax.numpy as jnp
from jax import jit, vmap
import numpy as np
import math

import os
import tifffile

import fastplotlib as fpl

%load_ext autoreload

# Specify the dataset in the below panel. See PMD documentation for easy ways to add support for your custom data formats. We provide default support for single, multipage tiff files. 

In [2]:
input_file = "../datasets/demoMovie.tif"
current_dataset = MultipageTiffDataset(input_file)

# Below panel specifies the parameters for running the method

In [3]:
#Default parameters (can be modified..)

pmd_params_dict = {
    'block_height':20,
    'block_width':20,
    'frames_to_init':5000,
    'background_rank':1,
    'max_consec_failures':1,
    'max_components':40,
}


#NOTE: this data folder will also contain the location of the TestData
block_height = pmd_params_dict['block_height']
block_width = pmd_params_dict['block_width'] 

block_sizes = [block_height, block_width]

max_consec_failures = pmd_params_dict['max_consec_failures']
frames_to_init = pmd_params_dict['frames_to_init']
background_rank = pmd_params_dict['background_rank'] 

###THESE PARAMS ARE NEVER MODIFIED
sim_conf = 5

max_components = pmd_params_dict['max_components']

corrector = None

frame_batch_size = 2000
pixel_batch_size = 10000
dtype="float32"

# Run PMD

In [4]:
U, R, s, V, std_img, mean_img, data_shape, data_order = localmd_decomposition(current_dataset, block_sizes, frames_to_init, \
                                max_components=max_components, background_rank = background_rank, sim_conf=sim_conf,\
                                 frame_batch_size=frame_batch_size,pixel_batch_size=pixel_batch_size, dtype=dtype, \
                                 num_workers=0, frame_corrector_obj = corrector, max_consec_failures=max_consec_failures)

current_video = PMDArray(U, R, s, V, data_shape, data_order, mean_img, std_img)

[23-11-29 16:28:59]: The batch size used is 2000
[23-11-29 16:28:59]: num workers for each dataloader is 0
[23-11-29 16:28:59]: Computing Video Statistics
[23-11-29 16:28:59]: Calculating mean and noise variance


An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


[23-11-29 16:29:00]: Finished mean and noise variance
[23-11-29 16:29:00]: WARNING: Specified using more frames than there are in the dataset.
[23-11-29 16:29:00]: WARNING: Initializing on less than 2000 frames, this will lead to limited benefits.
[23-11-29 16:29:00]: We are initializing on a total of 2000 frames
[23-11-29 16:29:00]: Running Simulations, block dimensions are 20 x 20 x 2000 
[23-11-29 16:29:03]: Loading Data
[23-11-29 16:29:03]: Obtaining blocks and running local SVD
[23-11-29 16:29:04]: Normalizing by weights
[23-11-29 16:29:04]: The total rank before pruning is 720
[23-11-29 16:29:04]: Performing rank pruning and orthogonalization for fast sparse regression.
[23-11-29 16:29:04]: After performing rank reduction, the updated rank is 240
[23-11-29 16:29:04]: Running sparse regression


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  2.70it/s]

[23-11-29 16:29:05]: Final reformat of data into complete SVD
[23-11-29 16:29:05]: Short matrix, using leftward SVD routine
[23-11-29 16:29:05]: Matrix decomposition completed


# Fastplotlib testing

In [5]:
current_video.shape
print(type(current_video))
current_video[0].shape

<class 'localmd.pmdarray.PMDArray'>


(60, 80)

In [28]:
iw = fpl.ImageWidget([current_video for i in range(4)], grid_plot_kwargs={"controllers": [[0, 1], [1,  0]]})
iw.show()

RFBOutputContext()

JupyterOutputContext(children=(JupyterWgpuCanvas(), IpywidgetToolBar(children=(Button(icon='expand-arrows-alt'…

In [30]:
from scipy.ndimage import gaussian_filter

In [32]:
iw.frame_apply = {0: lambda f: gaussian_filter(f, 5)}

In [ ]:
gp = fpl.GridPlot([

In [22]:
iw.gridplot.canvas.quality = 50

In [24]:
iw.window_funcs = None

In [7]:
iw.cmap = "gnuplot2"